In [1]:
import requests
import time
import pandas as pd
import numpy as np
import re
import json
import pdb
from bs4 import BeautifulSoup
import personal_keys
import ravelry_request_functions

In [2]:
ravelry_request_functions.get_favs_list_edited("katec125")

[777372,
 1002216,
 998422,
 877616,
 990541,
 966823,
 444597,
 888409,
 420814,
 790875,
 409979,
 538253,
 387775,
 994876,
 993218,
 975032,
 995685,
 731341,
 984587,
 384733,
 866947,
 932512,
 893880,
 986145,
 807636,
 936349,
 918043,
 907192,
 843846,
 866773,
 686988,
 966033,
 968150,
 869199,
 351173,
 824391,
 921518,
 843810,
 855203,
 925240,
 929139,
 570677,
 483117,
 639396,
 754362,
 824302,
 886241,
 561403,
 171113,
 57264,
 772466,
 497126,
 680396,
 868477,
 868478,
 497647,
 497647,
 497647,
 582257,
 377778,
 593997,
 838914,
 742300,
 858134,
 855568,
 763885,
 830861,
 853849,
 690209,
 843963,
 804284,
 853145,
 743018,
 758635,
 745500,
 847738,
 842474,
 658135,
 833778,
 841735,
 602732,
 833943,
 833941,
 606529,
 785079,
 814274,
 761738,
 829488,
 829043,
 825008,
 526155,
 752414,
 373662,
 412555,
 534321,
 580557,
 807394,
 536459,
 522775,
 466908,
 [172113,
  481187,
  593681,
  746797,
  676427,
  516187,
  651122,
  686917,
  679511,
  36436,
 

In [12]:
def get_favs_list(username):
    fav_list = []
    favs_url = 'https://api.ravelry.com/people/{}/favorites/list.json'.format(username)
#     following request returns json with two keys: favorites and paginator
#     for the following request, I've added the params page size, 
#     which is 100 (maximum size), and page number (first page in the following request)
    favs = requests.get(favs_url, auth = (personal_keys.username(),personal_keys.password()),
                        params={'page_size':100, 'page':1})
#     the following will add a list of all the pattern ids for the favorited patterns and turns them into a list, which is then appended to the fav_list defined at the beginning of the function
    for item in range(0,len(favs.json()['favorites'])):
        if favs.json()['favorites'][item]['favorited'] is not None:
            if 'pattern_id' in favs.json()['favorites'][item]['favorited'].keys():
                fav_list.append(favs.json()['favorites'][item]['favorited']['pattern_id'])
            elif 'id' in favs.json()['favorites'][item]['favorited'].keys():
                fav_list.append(favs.json()['favorites'][item]['favorited']['id'])

#     in the event the user has more than 100 favorites, 
# this loop will essentially go in and make a new request for the next page of likes, 
# this loop finishes when the page number (which starts at 2 because we've already requested page one above) 
# equals the last page
    if favs.json()['paginator']['page_count']>1:
#         page_number is 2 because first page is already in fav_list
        page_number = 2
#         loop to do successive requests for new pages of favorited pattern ids
        if favs.json()['paginator']['last_page'] > 5:
            last_page = 5
        else:
            last_page = favs.json()['paginator']['last_page']
        while page_number <= last_page:
            new_request_favs = requests.get(favs_url, auth = (personal_keys.username(),personal_keys.password()),
                        params={'page_size':100, 'page':page_number})
#             append the new list of favorites to the old list containing the first 100 favorites
            for item in range(0,len(new_request_favs.json()['favorites'])):
                if new_request_favs.json()['favorites'][item]['favorited'] is not None:
                    if 'pattern_id' in new_request_favs.json()['favorites'][item]['favorited'].keys():
                        fav_list.append(new_request_favs.json()['favorites'][item]['favorited']['pattern_id'])
                    elif 'id' in new_request_favs.json()['favorites'][item]['favorited'].keys():
                        fav_list.append(new_request_favs.json()['favorites'][item]['favorited']['id'])
#             add one to the page number so it will request the page following page 2, this will be done for as long as there are more pages to be requested
            page_number += 1
#     since the fav_list is currently a list of lists, the following function flattens the fav_list into one long list of ids
    return fav_list

In [22]:
kate_fav_list = get_favs_list('katec125')
edited_kate_fav_list = ravelry_request_functions.problem_children(kate_fav_list)
taken_out_favs = [p for p in kate_fav_list if str(p) not in edited_kate_fav_list]
len(taken_out_favs)


0

In [3]:
favs_url = 'https://api.ravelry.com/people/{}/favorites/list.json'.format('katec125')

favs = requests.get(favs_url, auth = (personal_keys.username(),personal_keys.password()),params={'page_size':100, 'page':1})

bad_favs_list =[favs.json()['favorites'][item]['favorited'] for item in range(0,len(favs.json()['favorites'])) if favs.json()['favorites'][item]['favorited']['id'] in taken_out_favs]

In [8]:
def get_friend_projs(username):    
    friend_list = friend_username_list(username)
    all_friend_projs = []
    for user in friend_list:
        all_friend_projs.append(get_project_list(user))        
    flat_list = [item for sublist in all_friend_projs for item in sublist]
    edited_flat_list = [item for item in flat_list if item is not None]
    return edited_flat_listbad_favs_list[5]['pattern_id']

497647

In [13]:
good_favs_list = [favs.json()['favorites'][item]['favorited'] for item in range(0,len(favs.json()['favorites'])) if favs.json()['favorites'][item]['favorited']['id'] not in taken_out_favs]

good_favs_list[5].keys()

dict_keys(['id', 'name', 'permalink', 'first_photo', 'designer', 'pattern_author', 'pattern_sources'])

In [10]:
patterns_url = 'https://api.ravelry.com/patterns/497647.json'
patterns = requests.get(patterns_url, auth = (personal_keys.username(),personal_keys.password()))
patterns.json()

.com/uploads/thepluckyknitter/239054377/Dotted_Rays_3_medium2.jpg',
    'small2_url': 'https://images4-f.ravelrycache.com/uploads/thepluckyknitter/239054377/Dotted_Rays_3_small2.jpg',
    'caption': None,
    'caption_html': None,
    'copyright_holder': 'thepluckyknitter'},
   {'id': 48104301,
    'sort_order': 9,
    'x_offset': 0,
    'y_offset': 0,
    'square_url': 'https://images4-g.ravelrycache.com/uploads/putyourflareon/289664144/image_square.jpg',
    'medium_url': 'https://images4-g.ravelrycache.com/uploads/putyourflareon/289664144/image_medium.jpg',
    'thumbnail_url': 'https://images4-g.ravelrycache.com/uploads/putyourflareon/289664144/image_thumbnail.jpg',
    'small_url': 'https://images4-g.ravelrycache.com/uploads/putyourflareon/289664144/image_small_best_fit.jpg',
    'flickr_url': None,
    'shelved_url': 'https://images4-f.ravelrycache.com/uploads/putyourflareon/289664144/image_shelved.jpg',
    'medium2_url': 'https://images4-f.ravelrycache.com/uploads/putyourflareo

In [2]:
def get_friend_favs(username):    
    friend_list = ravelry_request_functions.friend_username_list(username)
    all_friend_projs = []
    for user in friend_list:
        all_friend_projs.append(ravelry_request_functions.get_favs_list(user))   
    flat_list = [item for sublist in all_friend_projs for item in sublist]
    edited_flat_list = [item for item in flat_list if item is not None]
    return edited_flat_list

In [3]:
kate_friend_fav_list = ravelry_request_functions.get_friend_favs('katec125')

In [17]:
import importlib
importlib.reload(ravelry_request_functions)

<module 'ravelry_request_functions' from '/Users/KateChristensen/Desktop/Ravelry/ravelry_request_functions.py'>

In [4]:
friend_fav_attr = ravelry_request_functions.pattern_attr(kate_friend_fav_list)

In [10]:
edited_friend_favs = ravelry_request_functions.problem_children_large_scale(kate_friend_fav_list)

In [5]:
len(friend_fav_attr)

4607

In [8]:
len(kate_friend_fav_list)

6191

In [11]:
len(edited_friend_favs)

6191

In [12]:
kate_favs = ravelry_request_functions.get_favs_list('katec125')

In [28]:
l_of_l_patterns = [kate_friend_fav_list[i:i + 50] for i in range(0, len(kate_friend_fav_list), 50)]

len(l_of_l_patterns[122])


50

In [10]:
len(ravelry_request_functions.get_favs_list('lynng1124'))

1390

In [13]:
test_favs = get_favs_list('lynng1124')
len(test_favs)

500

In [9]:
fav_list = []
favs_url = 'https://api.ravelry.com/people/{}/favorites/list.json'.format("katec125")
favs = requests.get(favs_url, auth = (personal_keys.username(),personal_keys.password()),
                        params={'page_size':100, 'page':1})

favs.json()['favorites']

  'shelf_image_path': 'http://images4.ravelry.com/uploads/verybusymonkey/134708425/logolarge_shelved.jpg',
     'shelf_image_size': '153x150',
     'small_image_url': 'http://images4.ravelry.com/uploads/verybusymonkey/134708425/logolarge_square.jpg',
     'source_group_id': None,
     'stickies_count': 0,
     'store_id': 2292,
     'updated_at': '2014/05/31 15:40:37 -0400',
     'url': 'http://www.ravelry.com/stores/stephanie-etting-designs',
     'work_id': None,
     'notes': ''}]},
  'comment': None},
 {'created_at': '2018/04/19 19:31:27 -0400',
  'id': 240500953,
  'type': 'pattern',
  'tag_list': '',
  'favorited': {'id': 807394,
   'name': 'Rooted Mitts',
   'permalink': 'rooted-mitts',
   'first_photo': {'id': 72816233,
    'sort_order': 1,
    'x_offset': -5,
    'y_offset': -35,
    'square_url': 'https://images4-f.ravelrycache.com/uploads/verybusymonkey/499552245/Blob_square',
    'medium_url': 'https://images4-g.ravelrycache.com/uploads/verybusymonkey/499552245/Blob_medium'